In [ ]:
!pip install llama-cpp-python  # For local LLaMA 3 inference


In [1]:
!pip install faiss-cpu # for vector db
!pip install PyPDF2 # for parsing pdfs
!pip install pandas
!pip install scikit-learn  # For TfidfVectorizer, cosine distance
!pip install langchain # for text splitting
!pip install tqdm # for progress bar
!pip install numpy

from typing import Dict, Any, List, Tuple
from helpers_llama3_transformers import (
    DocumentProcessor,
    VectorStore,
    HybridSearcher,
    Document,
    get_completion,
    get_embedding
)
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 10.4 MB/s eta 0:00:00a 0:00:

In [5]:
def load_documents(file_path):
    # Load and chunk document
    text = DocumentProcessor.load_pdf(file_path)
    documents = DocumentProcessor.chunk_text(text)
    
    # Create and return vector store
    vector_store = VectorStore()  # HybridSearcher()
    vector_store.create_index(documents)
    print(f"Processed {len(documents)} chunks")
    
    return vector_store

vector_store = load_documents("WF_benefits_book.pdf")

Loading PDF pages:   0%|          | 0/458 [00:00<?, ?it/s]

Cleaning extracted text...
Splitting text into initial chunks...


Processing chunks:   0%|          | 0/2074 [00:00<?, ?chunk/s]

Final number of chunks: 2074
Creating new index...


Generating embeddings:   0%|          | 0/2074 [00:00<?, ?it/s]

Saved index with 2074 documents
Processed 2074 chunks


In [6]:
# query = "What is Wells Fargo?"
# vector_store.search(query, k=5)

[(Document(content='While reading this material, be aware that: • The plans are provided as a benefit to eligible employees, eligible former employees who have elected continuation coverage under COBRA (if applicable), and the respective eligible dependents of either of the above. Participation in these plans does not constitute a contract or guarantee of employment with Wells Fargo & Company or its subsidiaries or affiliates. Plan benefits depend on continued eligibility. • The name “Wells Fargo” as used throughout this document refers to “Wells Fargo & Company.”In case of any conflict between the SPDs in this Benefits Book or any other information provided and the official plan document, the official plan document governs', metadata=None),
  0.6765205043115782),
 (Document(content='Alternatively, Wells Fargo may, but is not required to, pay such fees and expenses directly. Wells Fargo may also advance amounts properly payable by the applicable plan or trust and then obtain reimbursem

In [8]:
def query(query, top_k=5):
    contents = vector_store.search(query, k=top_k)
    contents = '\n'.join([x[0].content for x in contents])
    print(contents)
    
    prompt = f""" You are a helpful assistant.
    You have ti question the contents below:
    question: {contents}
    query: {query}
    
    Given  your answer based on the contents. You can say no if there is nothing to answer.
    
    Answer:
    """
    
query("What benefits are listed by Wells Fargo?")

=== Page 1 === Benefits Book A guide to your Wells Fargo benefits Effective January 1, 2024 Use this guide to review eligibility information, plan coverage details, important notifications and disclosures, and more. This book is meant for eligible employees on U.S. payroll
Alternatively, Wells Fargo may, but is not required to, pay such fees and expenses directly. Wells Fargo may also advance amounts properly payable by the applicable plan or trust and then obtain reimbursement from the plan or trust for such advance. For benefits not funded through a trust, fees and expenses in curred in connection with the operation and administration of the benefits may be paid out of the assets of the applicable plan to the extent that it is legally permissible for such fees and expenses to be so paid. Alternatively, Wells Fargo may, but is not required to, pay such fees and expenses directly. Wells Fargo may also advance amounts properly payable by the applicable plan and then obtain reimbursement

In [ ]:
def get_relevant_chunks(
    question,
    vector_store,
    top_k = 3):
    """
    Retrieve relevant document chunks for a question
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of chunks to retrieve
    Returns:
        List of (document, relevance_score) tuples
    """
    return vector_store.search(question, k=top_k)

In [13]:
def get_relevant_chunks(
    question,
    vector_store,
    top_k = 3):
    """
    Retrieve relevant document chunks for a question
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of chunks to retrieve
    Returns:
        List of (document, relevance_score) tuples
    """
    return vector_store.search(question, k=top_k)

def generate_answer(
    question,
    relevant_chunks):
    """
    Generate an answer using retrieved chunks
    Args:
        question: User's question
        relevant_chunks: List of relevant documents and their scores
    Returns:
        Generated answer
    """
    # Format context from relevant chunks
    contexts = [
        f"[{i}] {doc.content}"
        for i, (doc, _) in enumerate(relevant_chunks, 1)
    ]
    
    prompt = f"""You are a helpful assistant.
    Answer the question based on the context. If the answer isn't in the context, say so.

    Question: {question}

    Context:
    {"/n/n".join(contexts)}

    Answer:"""
    
    return get_completion(prompt)

def query(
    question,
    vector_store,
    top_k = 10):
    """
    Process a question and return an answer with sources
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve
    Returns:
        Dictionary containing answer and sources
    """
    # Get relevant chunks
    relevant_chunks = get_relevant_chunks(question, vector_store, top_k)
    
    # Generate answer
    answer = generate_answer(question, relevant_chunks)
    
    # Format and return response
    return answer, relevant_chunks

question = "What is COBRA mentioned in the document?"
answer, relevant_chunks = query(question, vector_store, top_k=10)

print("\nAnswer:", answer)

# Print sources
print("\nSources:")
for i, source in enumerate(relevant_chunks, 1):
    print(f"\n[{i}] Relevance score: {relevant_chunks[i-1][1]:.2f}")
    print(f"Preview: {relevant_chunks[i-1][0].content}")

    


Answer: COBRA is the Consolidated Omnibus Budget Reconciliation Act of 1985. It allows continuation of group health coverage when it would otherwise end due to certain life events. 


Sources:

[1] Relevance score: 0.69
Preview: The Plan will offer COBRA continuation coverage to qualified beneficiaries only after the COBRA Administrator has been notified that a qualifying event has occurred

[2] Relevance score: 0.67
Preview: For more information about Medicare, visit https://www.medicare.gov/medicare-and-you . If you have questions Questions concerning your Plan or your COBRA continuation coverage rights should be addressed to the COBRA Administrator: BenefitConnect™ | COBRA at 1-877-29-COBRA (1-877-292-6272) (858-314-5108 International only). Representatives are available from Monday through Friday, 8:00 a.m. to 6:00 p.m. Central Time. You may also access https://cobra.ehr.com . For more information about your rights under the Employee Retirement Income Security Act of 1974, as amen

In [14]:
def get_relevant_chunks(
    question,
    vector_store,
    top_k = 3):
    """
    Retrieve relevant document chunks for a question
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of chunks to retrieve
    Returns:
        List of (document, relevance_score) tuples
    """
    return vector_store.search(question, k=top_k)

def generate_answer(
    question,
    relevant_chunks):
    """
    Generate an answer using retrieved chunks
    Args:
        question: User's question
        relevant_chunks: List of relevant documents and their scores
    Returns:
        Generated answer
    """
    # Format context from relevant chunks
    contexts = [
        f"[{i}] {doc.content}"
        for i, (doc, _) in enumerate(relevant_chunks, 1)
    ]
    
    prompt = f"""You are a helpful assistant.
    Answer the question based on the context. If the answer isn't in the context, say so.

    Question: {question}

    Context:
    {"/n/n".join(contexts)}

    Answer:"""
    
    return get_completion(prompt)

def query(
    question,
    vector_store,
    top_k = 10):
    """
    Process a question and return an answer with sources
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve
    Returns:
        Dictionary containing answer and sources
    """
    # Get relevant chunks
    relevant_chunks = get_relevant_chunks(question, vector_store, top_k)
    
    # Generate answer
    answer = generate_answer(question, relevant_chunks)
    
    # Format and return response
    return answer, relevant_chunks

question = "How does Wells Fargos benefits compare to Vanguard?"
answer, relevant_chunks = query(question, vector_store, top_k=10)

print("\nAnswer:", answer)

# Print sources
print("\nSources:")
for i, source in enumerate(relevant_chunks, 1):
    print(f"\n[{i}] Relevance score: {relevant_chunks[i-1][1]:.2f}")
    print(f"Preview: {relevant_chunks[i-1][0].content}")

    


Answer: The provided text does not contain information about Wells Fargo's benefits compared to Vanguard. 


Sources:

[1] Relevance score: 0.70
Preview: Alternatively, Wells Fargo may, but is not required to, pay such fees and expenses directly. Wells Fargo may also advance amounts properly payable by the applicable plan or trust and then obtain reimbursement from the plan or trust for such advance. For benefits not funded through a trust, fees and expenses in curred in connection with the operation and administration of the benefits may be paid out of the assets of the applicable plan to the extent that it is legally permissible for such fees and expenses to be so paid. Alternatively, Wells Fargo may, but is not required to, pay such fees and expenses directly. Wells Fargo may also advance amounts properly payable by the applicable plan and then obtain reimbursement from the plan for such advance. How to enroll General information You may enroll online in the benefit plan options fo

In [18]:
def rerank_chunks(
    question,
    chunks,
    top_k = 3):
    """
    Rerank chunks using semantic similarity
    """
    question_embedding = get_embedding(question)
    
    reranked = []
    for doc, initial_score in chunks:
        # Get chunk embedding
        chunk_embedding = get_embedding(doc.content)
        
        # Calculate semantic similarity
        similarity = 1 - cosine_similarity(
            [question_embedding],
            [chunk_embedding]
        )[0][0]
        
        # Combine with initial score
        final_score = (initial_score + similarity) / 2
        reranked.append((doc, final_score))
    
    # Sort by final score and return top_k
    return sorted(reranked, key=lambda x: x[1], reverse=True)[:top_k]

def generate_answer_adv(
    question,
    relevant_chunks):
    """
    Generate answer with improved prompt and source attribution
    """
    # Format contexts with relevance scores
    contexts = [
        f"[{i}] (Relevance: {score:.2f}) {doc.content}"
        for i, (doc, score) in enumerate(relevant_chunks, 1)
    ]

    prompt = f"""You are a precise assistant that always cites sources.
    Answer the question using only the provided contexts. Cite context numbers [N].
    
    Question: {question}
    
    Contexts:
    {"/n/n".join(contexts)}
    
    Remember:
    - Use only provided contexts
    - Cite your sources
    - Be clear if information is missing
    
    Answer:"""
            
    return get_completion(prompt)

def verify_answer(
    question,
    answer,
    chunks):
    """
    Verify answer against source chunks
    """
    # Create verification prompt
    contexts = [doc.content for doc, _ in chunks]

    prompt = f"""You are a fact-checking assistant.
    Analyze the following answer's accuracy based on the provided contexts.
    
    Question: {question}
    Answer: {answer}
    
    Contexts: 
    {"/n/n".join(contexts)}
    
    Verify the following:
    1. Is the answer factually accurate based on the contexts?
    2. Does it fully answer the question?
    3. Are there any unsupported claims?
    4. What is your confidence in the answer's accuracy (high/medium/low)?
    
    Provide a brief verification summary addressing these points."""
     
    return get_completion(prompt)

def query2(
    question,
    vector_store,
    top_k = 10):
    """
    Process a question and return an answer with sources
    Args:
        question: User's question
        vector_store: Indexed document store
        top_k: Number of relevant chunks to retrieve
    Returns:
        Dictionary containing answer and sources
    """
    # Get relevant chunks
    relevant_chunks = get_relevant_chunks(question, vector_store, top_k * 2)

    # Rerank chunks
    relevant_chunks_reranked = rerank_chunks(question, relevant_chunks, top_k)
    
    # Generate answer
    answer = generate_answer_adv(question, relevant_chunks)

    # Verify answer
    verification = verify_answer(question, answer, relevant_chunks_reranked)
    
    # Format and return response
    return answer, verification, relevant_chunks

question = "What is COBRA mentioned in the document?"
answer, verification, relevant_chunks = query2(question, vector_store, top_k=10)

print("\nAnswer:", answer)

print("\nVerification:", verification)

# Print sources
print("\nSources:")
for i, source in enumerate(relevant_chunks, 1):
    print(f"\n[{i}] Relevance score: {relevant_chunks[i-1][1]:.2f}")
    print(f"Preview: {relevant_chunks[i-1][0].content}")



Answer: COBRA is the Consolidated Omnibus Budget Reconciliation Act of 1985. [5] It is a federal law that allows employees and their families to continue group health coverage when it would otherwise end because of certain life events. [5, 7] These events are called "qualifying events." [7]  Specific qualifying events are listed later in the notice. [7]  COBRA continuation coverage can become available to you and other members of your family when group health coverage would otherwise end. [5]  COBRA continuation coverage is a continuation of Plan coverage when it would otherwise end because of certain life events. [7]  COBRA continuation coverage is a temporary continuation of coverage that generally lasts for 18 months due to employment termination or reduction of hours of work. [13]  Certain qualifying events may permit a beneficiary to receive a maximum of 36 months of coverage. [13]  There are also two ways in which this 18-month period of COBRA continuation coverage can be extend